In [2]:
%pip install diffusers transformers accelerate peft
%pip install --quiet kagglehub
%pip install -q bitsandbytes


In [ ]:
import torch
from torch.utils.data import Dataset
from PIL import Image
import os
import json
import random
import zipfile
import numpy as np
from diffusers import StableDiffusionXLPipeline, UNet2DConditionModel, DDPMScheduler, AutoencoderKL
from peft import LoraConfig, get_peft_model
from transformers import CLIPTextModel, CLIPTextModelWithProjection, CLIPTokenizer
from accelerate import Accelerator
from tqdm import tqdm
import kagglehub
from huggingface_hub import login

# Set environment variable to handle memory fragmentation
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Étape 0 : Authentification Hugging Face
hf_token = os.getenv("HF_TOKEN")
if not hf_token:
    from getpass import getpass
    hf_token = getpass("Entrez votre token Hugging Face : ")
login(hf_token)
print("Authentification HF réussie.")

# Étape 1 : Télécharger le Dataset Kaggle
dataset_name = "fatihkgg/ecommerce-product-images-18k"
dataset_path = kagglehub.dataset_download(dataset_name, force_download=False)
print(f"Dataset downloaded to: {dataset_path}")

# Extraction ZIP
top_level_contents = os.listdir(dataset_path)
print(f"Top-level contents: {top_level_contents}")
zip_files = [f for f in top_level_contents if f.lower().endswith('.zip')]
if zip_files:
    for zip_file in zip_files:
        zip_path = os.path.join(dataset_path, zip_file)
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(dataset_path)
        print(f"Extracted {zip_path}")
        os.remove(zip_path)
    top_level_contents = os.listdir(dataset_path)

root_folder = None
if len(top_level_contents) == 1 and os.path.isdir(os.path.join(dataset_path, top_level_contents[0])):
    root_folder = top_level_contents[0]
train_dir = os.path.join(dataset_path, root_folder if root_folder else "", "train")
if not os.path.exists(train_dir):
    raise FileNotFoundError(f"Train dir not found at {train_dir}.")
categories = [d for d in os.listdir(train_dir) if os.path.isdir(os.path.join(train_dir, d))]
print(f"Categories: {categories}")

# Étape 2 : Hyperparamètres SDXL (PLUS SIMPLE)
model_id = "stabilityai/stable-diffusion-xl-base-1.0"
output_dir = "fine_tuned_sdxl_ecommerce"
resolution = 512  # Résolution modeste
train_batch_size = 1
num_epochs = 3  # Réduit pour tester
learning_rate = 1e-4
lora_rank = 4  # Très petit pour économiser VRAM
gradient_accumulation_steps = 4

# Étape 3 : Dataset
class KaggleEcommerceDataset(Dataset):
    def __init__(self, root_dir, tokenizer, tokenizer_2, resolution, categories, num_images=1000):
        self.root_dir = root_dir
        self.tokenizer = tokenizer
        self.tokenizer_2 = tokenizer_2
        self.resolution = resolution
        self.image_files = []
        self.captions = {}

        colors = ['rouge', 'bleu', 'vert', 'noir', 'blanc', 'jaune']
        styles = ['moderne', 'classique', 'sportif', 'élégant', 'casual']

        for category in categories:
            cat_dir = os.path.join(self.root_dir, category)
            for img_file in os.listdir(cat_dir):
                if img_file.lower().endswith(('.jpg', '.png', '.jpeg')):
                    img_path = os.path.join(cat_dir, img_file)
                    self.image_files.append(img_path)
                    color = random.choice(colors)
                    style = random.choice(styles)
                    caption = f"produit e-commerce {category}, {color}, {style}, haute qualité"
                    self.captions[img_path] = caption

        # Save generated_captions.json to a writable directory, e.g., /content/
        # The `root_dir` (/kaggle/input/...) is read-only, causing the OSError.
        output_caption_path = os.path.join("/content/", "generated_captions.json")
        with open(output_caption_path, "w") as f:
            json.dump(self.captions, f)

        print(f"Dataset créé : {len(self.image_files)} images. Captions saved to {output_caption_path}")

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = self.image_files[idx]
        image = Image.open(img_path).convert("RGB")
        image = image.resize((self.resolution, self.resolution), Image.LANCZOS)

        # Conversion correcte PIL -> tensor
        image = np.array(image)
        image = torch.from_numpy(image).permute(2, 0, 1).float() / 127.5 - 1.0

        caption = self.captions[img_path]

        # Tokenization pour les 2 encodeurs CLIP de SDXL
        text_input_1 = self.tokenizer(
            caption, padding="max_length", max_length=77, truncation=True, return_tensors="pt"
        )
        text_input_2 = self.tokenizer_2(
            caption, padding="max_length", max_length=77, truncation=True, return_tensors="pt"
        )

        return {
            "pixel_values": image,
            "input_ids_1": text_input_1["input_ids"].squeeze(0),
            "input_ids_2": text_input_2["input_ids"].squeeze(0),
        }

# Étape 4 : Charger le Modèle SDXL
print("Loading tokenizers...")
tokenizer = CLIPTokenizer.from_pretrained(model_id, subfolder="tokenizer", token=hf_token)
tokenizer_2 = CLIPTokenizer.from_pretrained(model_id, subfolder="tokenizer_2", token=hf_token)

print("Loading text encoders...")
text_encoder = CLIPTextModel.from_pretrained(
    model_id, subfolder="text_encoder", token=hf_token, torch_dtype=torch.float16
)
text_encoder_2 = CLIPTextModelWithProjection.from_pretrained(
    model_id, subfolder="text_encoder_2", token=hf_token, torch_dtype=torch.float16
)
text_encoder.requires_grad_(False)
text_encoder_2.requires_grad_(False)

print("Loading VAE...")
vae = AutoencoderKL.from_pretrained(
    model_id, subfolder="vae", token=hf_token, torch_dtype=torch.float16
)
vae.requires_grad_(False)

print("Loading UNet...")
unet = UNet2DConditionModel.from_pretrained(
    model_id, subfolder="unet", token=hf_token, torch_dtype=torch.float16
)

# Enable gradient checkpointing to save memory
unet.enable_gradient_checkpointing()

print("Loading scheduler...")
scheduler = DDPMScheduler.from_pretrained(model_id, subfolder="scheduler", token=hf_token)

# Appliquer LoRA sur UNet
print("Applying LoRA to UNet...")
lora_config = LoraConfig(
    r=lora_rank,
    lora_alpha=lora_rank * 2,
    target_modules=["to_k", "to_q", "to_v", "to_out.0"],
    lora_dropout=0.05,
    bias="none",
)
unet = get_peft_model(unet, lora_config)
unet.print_trainable_parameters()

# Initialiser Accelerator
accelerator = Accelerator(
    gradient_accumulation_steps=gradient_accumulation_steps,
    mixed_precision="fp16"
)

# Optimizer
optimizer = torch.optim.AdamW(
    [p for p in unet.parameters() if p.requires_grad],
    lr=learning_rate
)

# Dataset et Dataloader
dataset = KaggleEcommerceDataset(train_dir, tokenizer, tokenizer_2, resolution, categories, num_images=100)
dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size=train_batch_size,
    shuffle=True,
    num_workers=0  # Évite les problèmes multiprocessing
)

lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=num_epochs * len(dataloader)
)

# Préparer avec Accelerator
unet, optimizer, dataloader, lr_scheduler = accelerator.prepare(
    unet, optimizer, dataloader, lr_scheduler
)

# Déplacer les modèles figés
text_encoder = text_encoder.to(accelerator.device)
text_encoder_2 = text_encoder_2.to(accelerator.device)
vae = vae.to(accelerator.device)

# Mode training
unet.train()
text_encoder.eval()
text_encoder_2.eval()
vae.eval()

# Étape 5 : Boucle d'Entraînement
print("Starting training...")
progress_bar = tqdm(range(num_epochs * len(dataloader)), desc="Training")

for epoch in range(num_epochs):
    for step, batch in enumerate(dataloader):
        with accelerator.accumulate(unet):
            # Récupérer les données
            pixel_values = batch["pixel_values"].to(accelerator.device)
            input_ids_1 = batch["input_ids_1"].to(accelerator.device)
            input_ids_2 = batch["input_ids_2"].to(accelerator.device)

            # Encoder images -> latents
            with torch.no_grad():
                latents = vae.encode(pixel_values.half()).latent_dist.sample()
                latents = latents * vae.config.scaling_factor
                latents = latents.float()  # Retour en float32 pour training

            # Encoder le texte avec les 2 encodeurs CLIP
            with torch.no_grad():
                # Premier encodeur
                encoder_output_1 = text_encoder(input_ids_1, output_hidden_states=True)
                text_embeds_1 = encoder_output_1.hidden_states[-2]

                # Deuxième encodeur (avec projection)
                encoder_output_2 = text_encoder_2(input_ids_2, output_hidden_states=True)
                text_embeds_2 = encoder_output_2.hidden_states[-2]
                pooled_embeds = encoder_output_2.text_embeds

                # Concaténer les embeddings
                text_embeds = torch.cat([text_embeds_1, text_embeds_2], dim=-1)

                # Convertir en float32
                text_embeds = text_embeds.float()
                pooled_embeds = pooled_embeds.float()

            # Ajouter du bruit
            noise = torch.randn_like(latents)
            timesteps = torch.randint(
                0, scheduler.config.num_train_timesteps, (latents.shape[0],),
                device=accelerator.device
            ).long()
            noisy_latents = scheduler.add_noise(latents, noise, timesteps)

            # Prédiction du bruit avec UNet
            # SDXL nécessite added_cond_kwargs pour pooled_embeds et time_ids corrects
            add_time_ids = torch.tensor([resolution, resolution, 0, 0, resolution, resolution], dtype=torch.float32, device=accelerator.device)
            added_cond_kwargs = {"text_embeds": pooled_embeds, "time_ids": add_time_ids.repeat(latents.shape[0], 1)}

            model_pred = unet(
                noisy_latents,
                timesteps,
                encoder_hidden_states=text_embeds,
                added_cond_kwargs=added_cond_kwargs,
                return_dict=False
            )[0]

            # Calcul de la loss
            loss = torch.nn.functional.mse_loss(model_pred, noise)

            # Backward
            accelerator.backward(loss)

            if accelerator.sync_gradients:
                accelerator.clip_grad_norm_(unet.parameters(), 1.0)

            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()

        progress_bar.update(1)

        if step % 10 == 0 and accelerator.is_main_process:
            print(f"Epoch {epoch}, Step {step}, Loss: {loss.item():.4f}")

        # Nettoyer la mémoire plus fréquemment
        if step % 20 == 0:
            torch.cuda.empty_cache()

print("Training complete!")

# Étape 6 : Sauvegarder
if accelerator.is_main_process:
    print("Saving model...")
    os.makedirs(output_dir, exist_ok=True)

    unwrapped_unet = accelerator.unwrap_model(unet)
    unwrapped_unet.save_pretrained(os.path.join(output_dir, "unet_lora"))

    print(f"Model saved to {output_dir}")

# Étape 7 : Inférence
print("Running inference...")
pipeline = StableDiffusionXLPipeline.from_pretrained(
    model_id,
    unet=accelerator.unwrap_model(unet),
    torch_dtype=torch.float16,
    token=hf_token
)
pipeline = pipeline.to("cuda")

prompt = "produit e-commerce t-shirt, bleu, moderne, haute qualité"
image = pipeline(
    prompt,
    num_inference_steps=30,
    guidance_scale=7.5,
    height=resolution,
    width=resolution
).images[0]

image.save("generated_ecommerce_product.png")
print("✅ Image générée : generated_ecommerce_product.png")

Authentification HF réussie.
Using Colab cache for faster access to the 'ecommerce-product-images-18k' dataset.
Dataset downloaded to: /kaggle/input/ecommerce-product-images-18k
Top-level contents: ['ECOMMERCE_PRODUCT_IMAGES']
Categories: ['ELECTRONICS', 'PET_SUPPLIES', 'BABY_PRODUCTS', 'SPORTS_OUTDOOR', 'HOME_KITCHEN_TOOLS', 'HOBBY_ARTS_STATIONERY', 'BEAUTY_HEALTH', 'CLOTHING_ACCESSORIES_JEWELLERY', 'GROCERY']
Loading tokenizers...
Loading text encoders...
Loading VAE...
Loading UNet...
Loading scheduler...
Applying LoRA to UNet...
trainable params: 5,806,080 || all params: 2,573,269,764 || trainable%: 0.2256
Dataset créé : 13992 images. Captions saved to /content/generated_captions.json
Starting training...



Training:   0%|          | 86/41976 [04:37<37:31:46,  3.23s/it]

Training:   0%|          | 1/41976 [00:02<25:07:37,  2.16s/it]

Epoch 0, Step 0, Loss: nan



Training:   0%|          | 11/41976 [00:17<17:00:55,  1.46s/it]

Epoch 0, Step 10, Loss: nan


...


Training:   5%|▌         | 2121/41976 [51:19<15:42:18,  1.42s/it]

Epoch 0, Step 2120, Loss: nan



Training:   5%|▌         | 2131/41976 [51:34<17:01:46,  1.54s/it]

Epoch 0, Step 2130, Loss: nan



Training:   5%|▌         | 2132/41976 [51:36<17:56:57,  1.62s/it]